In [7]:
import sourmash
import screed
import os
import re

from tqdm import tqdm

from kmer_utils import get_encoded_kmer_hashvals

# from sourmash.search import GatherDatabases, PrefetchResult, calculate_prefetch_info, format_bp
# from sourmash.logging import notify, error, print_results, set_quiet

## cd to botryllus protein directory

In [8]:
cd /Users/olgabot/botryllus/data/botryllus-proteins/

/Users/olgabot/botryllus-data/data/botryllus-proteins


## Make singleton signautre - one signature per sample

In [23]:
botryllus_sigfile = 'Bs_proteins.fa.hp.k24.scale5.sig'

In [ ]:
# ! sourmash sketch protein --output $botryllus_sigfile --singleton --hp -p k=24,scaled=5,abund Bs_proteins.fa.gz

## Make non-singleton aggregated signature

In [9]:
botryllus_sigfile = 'Bs_proteins.fa.hp.k24.scale5.aggregated.sig'

In [11]:
! time sourmash sketch protein --output $botryllus_sigfile --hp -p k=24,scaled=5,abund Bs_proteins.fa.gz


== This is sourmash version 4.4.0. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==

computing signatures for files: Bs_proteins.fa.gz
Computing a total of 1 signature(s) for each input.
... reading sequences from Bs_proteins.fa.gz
calculated 1 signatures for 72617 sequences in Bs_proteins.fa.gz
saved 1 signature(s) to 'Bs_proteins.fa.hp.k24.scale5.aggregated.sig'. Note: signature license is CC0.
sourmash sketch protein --output Bs_proteins.fa.hp.k24.scale5.aggregated.sig   120.84s user 1.21s system 99% cpu 2:02.96 total


In [40]:
! time /Users/olgabot/opt/miniconda3/envs/sourmash-dev/bin/sourmash sketch protein --output $botryllus_sigfile --hp -p k=24,scaled=5,abund Bs_proteins.fa.gz

sourmash: error: argument cmd: invalid choice: 'sketch' (choose from 'categorize', 'compare', 'compute', 'dump', 'gather', 'import_csv', 'index', 'info', 'migrate', 'multigather', 'plot', 'sbt_combine', 'search', 'watch', 'lca', 'sig', 'signature', 'storage')
/Users/olgabot/opt/miniconda3/envs/sourmash-dev/bin/sourmash sketch protein    0.25s user 0.06s system 87% cpu 0.356 total


In [35]:
! sourmash sig describe $botryllus_sigfile


== This is sourmash version 4.4.0. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==

---
signature filename: Bs_proteins.fa.hp.k24.scale5.aggregated.sig
signature: ** no name **
source file: Bs_proteins.fa.gz
md5: f06096cfbbc24ea0a9b53c3f1385ee2d
k=24 molecule=hp num=0 scaled=5 seed=42 track_abundance=1
size: 1499641
sum hashes: 3061757
signature license: CC0

loaded 1 signatures total, from 1 filesa.hp.k24.scale5.aggregated.sig'


## Get kmers for each hashval

In [ ]:
! sourmash sig kmers

In [29]:
# ! sourmash sig describe $mouse_sigfile | head -n 20


== This is sourmash version 4.2.2. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==

---
signature filename: /Users/olgabot/botryllus/data/gencode/M28/gencode.vM28.basic.annotation.protein.fa.hp.k24.scale5.sig
signature: ENSMUST00000070533.5 gene_id=ENSMUSG00000051951.6;transcript_id=ENSMUST00000070533.5;gene_type=protein_coding;gene_name=Xkr4;transcript_type=protein_coding;transcript_name=Xkr4-201;level=2;protein_id=ENSMUSP00000070648.5;transcript_support_level=1;mgi_id=MGI:3528744;tag=basic,appris_principal_1,CCDS;ccdsid=CCDS14803.1;havana_gene=OTTMUSG00000026353.2;havana_transcript=OTTMUST00000065166.1
source file: gencode.vM28.basic.annotation.protein.fa
md5: 2d08227f0c26a774c7e4134e4c288e3d
k=24 molecule=hp num=0 scaled=5 seed=42 track_abundance=1
size: 121
signature license: CC0

---
signature filename: /Users/olgabot/botryllus/data/gencode/M28/gencode.vM28.basic.annotation.protein.fa.hp.k24.scale5.sig
signature: ENSMUST00000208660.2 gene_id=ENSMUSG000000259

In [25]:
ls -lha

total 520280
drwxr-xr-x  21 olgabot  staff   672B Nov 11 07:35 ./
drwxr-xr-x  14 olgabot  staff   448B Apr 16  2022 ../
-rw-r--r--   1 olgabot  staff    18M Jun 22  2021 Bs_proteins.fa
-rw-r--r--   1 olgabot  staff   9.9M Jun 22  2021 Bs_proteins.fa.gz
-rw-r--r--   1 olgabot  staff    53B Nov 11 07:35 Bs_proteins.fa.hp.k24.scale5.aggregated.kmers.csv
-rw-r--r--   1 olgabot  staff    31M Nov 11 07:17 Bs_proteins.fa.hp.k24.scale5.aggregated.sig
-rw-r--r--   1 olgabot  staff    85M Apr 16  2022 Bs_proteins.fa.hp.k24.scale5.sig
-rw-r--r--   1 olgabot  staff   6.4M May 21 10:40 Bs_proteins.fa.hp.k24.scale5.sig.csv
-rw-r--r--   1 olgabot  staff   1.1K Dec 17  2021 bhf_isoform1.fa.hp.k24.scale5.sig
-rw-r--r--   1 olgabot  staff   698B Dec  3  2021 bhf_isoform1.sig
-rw-r--r--   1 olgabot  staff   986B Dec 17  2021 bhf_isoform2.fa.hp.k24.scale5.sig
-rw-r--r--   1 olgabot  staff   656B Dec  3  2021 bhf_isoform2.sig
-rw-r--r--   1 olgabot  staff   503B Dec  3  2021 bhf_isoforms.fa
-rw-r--r--   1 

In [39]:
sourmash.load_one_signature?

Signature:
sourmash.load_one_signature(
    data,
    ksize=None,
    select_moltype=None,
    ignore_md5sum=False,
)
Docstring: <no docstring>
File:      ~/opt/miniconda3/envs/botryllus/lib/python3.10/site-packages/sourmash/signature.py
Type:      function


In [49]:
! /Users/olgabot/opt/miniconda3/envs/botryllus/bin/python \
    ~/code/botryllus-mhc/notebooks/sig2kmer.py \
    --output-kmers Bs_proteins.fa.hp.k24.scale5.aggregated.kmers.csv \
    --input-is-protein --hp -k 72 --no-dna $botryllus_sigfile Bs_proteins.fa.gz

### Make sure the file has output

In [45]:
ls -lha Bs_proteins.fa.hp.k24.scale5.aggregated.kmers.csv

-rw-r--r--  1 olgabot  staff    31M Nov 11 08:53 Bs_proteins.fa.hp.k24.scale5.aggregated.kmers.csv


In [46]:
! head Bs_proteins.fa.hp.k24.scale5.aggregated.kmers.csv

kmer_in_sequence,kmer_in_alphabet,hashval,read_name
